In [4]:
!pip install duckdb

In [5]:
import duckdb

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from py_duckdb.similarity_join import tokenizers

In [8]:
con = duckdb.connect(database=':memory:')

# Source data

In [94]:
con.execute("drop table if exists src1").execute(
    "CREATE TABLE src1 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S1_clean_.csv'"
).execute("select * from src1").fetchall()

[('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'),
 ('S1_8', 'olivia hobson 19760812'),
 ('S1_9', 'michael lierach 19360816'),
 ('S1_10', 'elisabett domiten 19081008'),
 ('S1_11', 'genoveffa hylander 19071008')]

In [95]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S2_clean_.csv'"
).execute("select * from src2").fetchall()

[('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 19321129'),
 ('S2_4', 'olivia hobson 19760812'),
 ('S2_5', 'joshua green 19010219'),
 ('S2_6', 'charlotte hyland 19340909'),
 ('S2_7', 'elisabet domitienn 19071008')]

In [96]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S3_clean_.csv'"
).execute("select * from src3").fetchall()

[('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 19760812'),
 ('S3_7', 'joshua green 19790110'),
 ('S3_8', 'keely clarke 19050410'),
 ('S3_9', 'joshua morriosn 19101123'),
 ('S3_11', 'genovefa hyllande 19071008')]

In [138]:
con.execute("drop view if exists srcall").execute(
    "create view srcall as "
    "select * from src1 "
    "union "
    "select * from src2 "
    "union "
    "select * from src3 "
).execute("select * from srcall").fetchall()

[('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 19760812'),
 ('S3_7', 'joshua green 19790110'),
 ('S3_8', 'keely clarke 19050410'),
 ('S3_9', 'joshua morriosn 19101123'),
 ('S3_11', 'genovefa hyllande 19071008'),
 ('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'),
 ('S1_8', 'olivia hobson 19760812'),
 ('S1_9', 'michael lierach 19360816'),
 ('S1_10', 'elisabett domiten 19081008'),
 ('S1_11', 'genoveffa hylander 19071008'),
 ('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 193

# Function call

In [166]:
# function args
l_table = 'srcall'
r_table = 'srcall'
l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
#tokenizer = tokenizers.WordsTokzr()
tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'

In [167]:
# Works for both self-join and generic join
tokzr_query = ''

if isinstance(tokenizer, tokenizers.WordsTokzr):
    separators = tokenizer.get_info()
    tokzr_query = (f"select src, rid, len(tks) as rlen, lower(unnest(tks)) as token "
                   "from ( "
                   f"select distinct src, rid, str_split_regex(val, {separators}) as tks """
                   f"from input "
                   ") ")
elif isinstance(tokenizer, tokenizers.QGramsTokzr):
    q = tokenizer.get_info()
    tokzr_query = (f"select distinct src, rid, rlen "
                   f", substring(concat(repeat('#', {q} - 1), "
                   f"lower(val), "
                   f"repeat('#',{q} - 1)),"
                   f"x, {q}) as token "
                   "from ("
                   f"select *, len(val) + {q} - 1 as rlen, unnest(generate_series(1, rlen)) as x "
                   f"from input "
                   ")")

tokzr_query

"select distinct src, rid, rlen , substring(concat(repeat('#', 3 - 1), lower(val), repeat('#',3 - 1)),x, 3) as token from (select *, len(val) + 3 - 1 as rlen, unnest(generate_series(1, rlen)) as x from input )"

In [168]:
con.execute("drop view if exists input")
if l_table == r_table: # self-join
    con.execute(
        "create view input as "
        f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
        f"from '{l_table}' "
    )
else: # generic join
    con.execute(
        "create view input as "
        f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
        f"from '{l_table}' "
        "union "
        f"select '{r_table}' as src, {r_key_attr} as rid, {r_join_attr} as val "
        f"from '{r_table}' "
    )
con.execute("select * from input").fetchall()

[('srcall', 'S3_0', 'emmerson loyck 19211129'),
 ('srcall', 'S3_1', 'michel wuchatsch 19190110'),
 ('srcall', 'S3_3', 'liersch michael 19360816'),
 ('srcall', 'S3_4', 'charlotte hyland 19460401'),
 ('srcall', 'S3_5', 'braedon schuetz 19440909'),
 ('srcall', 'S3_6', 'olivia hobson 19760812'),
 ('srcall', 'S3_7', 'joshua green 19790110'),
 ('srcall', 'S3_8', 'keely clarke 19050410'),
 ('srcall', 'S3_9', 'joshua morriosn 19101123'),
 ('srcall', 'S3_11', 'genovefa hyllande 19071008'),
 ('srcall', 'S2_0', 'braecon schuetz 19440909'),
 ('srcall', 'S2_1', 'alexandra grosvenor 19930305'),
 ('srcall', 'S2_2', 'michael liersch 19360816'),
 ('srcall', 'S2_3', 'emmeron loyk 19321129'),
 ('srcall', 'S2_4', 'olivia hobson 19760812'),
 ('srcall', 'S2_5', 'joshua green 19010219'),
 ('srcall', 'S2_6', 'charlotte hyland 19340909'),
 ('srcall', 'S2_7', 'elisabet domitienn 19071008'),
 ('srcall', 'S1_0', 'joshua morrison 19101123'),
 ('srcall', 'S1_1', 'jordan white 19371126'),
 ('srcall', 'S1_2', 'emmers

In [169]:
# Works for both self-join and generic join
con.execute("drop view if exists tokens").execute(
    "create view tokens as " + tokzr_query
).execute("select * from tokens").fetchall()

[('srcall', 'S3_0', 25, '##e'),
 ('srcall', 'S3_0', 25, '#em'),
 ('srcall', 'S3_0', 25, 'emm'),
 ('srcall', 'S3_0', 25, 'mme'),
 ('srcall', 'S3_0', 25, 'mer'),
 ('srcall', 'S3_0', 25, 'ers'),
 ('srcall', 'S3_0', 25, 'rso'),
 ('srcall', 'S3_0', 25, 'son'),
 ('srcall', 'S3_0', 25, 'on '),
 ('srcall', 'S3_0', 25, 'n l'),
 ('srcall', 'S3_0', 25, ' lo'),
 ('srcall', 'S3_0', 25, 'loy'),
 ('srcall', 'S3_0', 25, 'oyc'),
 ('srcall', 'S3_0', 25, 'ck '),
 ('srcall', 'S3_0', 25, 'k 1'),
 ('srcall', 'S3_0', 25, ' 19'),
 ('srcall', 'S3_0', 25, '192'),
 ('srcall', 'S3_0', 25, '921'),
 ('srcall', 'S3_0', 25, '211'),
 ('srcall', 'S3_0', 25, '111'),
 ('srcall', 'S3_0', 25, '112'),
 ('srcall', 'S3_0', 25, '129'),
 ('srcall', 'S3_0', 25, '29#'),
 ('srcall', 'S3_1', 27, '##m'),
 ('srcall', 'S3_1', 27, '#mi'),
 ('srcall', 'S3_1', 27, 'mic'),
 ('srcall', 'S3_1', 27, 'ich'),
 ('srcall', 'S3_1', 27, 'che'),
 ('srcall', 'S3_1', 27, 'hel'),
 ('srcall', 'S3_1', 27, 'el '),
 ('srcall', 'S3_1', 27, 'l w'),
 ('srcal

In [170]:
t = threshold # shorter name

# Generic join

In [159]:
# brute force join, to get the gold standard
con.execute("drop view if exists bfjoin").execute(
    "create view bfjoin as "
    "select r1.rid as rid1, r2.rid as rid2, count(*) as overlap "
    "from tokens as r1, tokens as r2 "
    "where r1.token = r2.token "
    #"and r1.rid < r2.rid "
    f"and r1.src = '{l_table}' and r2.src = '{r_table}' "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
)

In [160]:
con.execute("select * from bfjoin").fetchall()

[('S1_3', 'S2_1', 20),
 ('S1_5', 'S2_3', 16),
 ('S1_6', 'S2_0', 17),
 ('S1_8', 'S2_4', 24),
 ('S1_9', 'S2_2', 23),
 ('S1_10', 'S2_7', 20)]

## Document frequency

In [161]:
con.execute("drop view if exists full_outer_df").execute(
    "create view full_outer_df as "
    "select s1.token as tk1, s1.df as df1, s2.token as tk2, s2.df as df2 "
    "from ("
    "SELECT token, count(*) AS df "
    "FROM tokens "
    f"where src = '{l_table}' "
    "GROUP BY token "
    ") as s1 "
    "full outer join ("
    "SELECT token, count(*) AS df "
    "FROM tokens "
    f"where src = '{r_table}' "
    "GROUP BY token "
    ") as s2 "
    "on s1.token = s2.token"
).execute("select * from full_outer_df").fetchall()

[('##j', 3, '##j', 1),
 ('#jo', 3, '#jo', 1),
 ('jos', 2, 'jos', 1),
 ('osh', 2, 'osh', 1),
 ('shu', 2, 'shu', 1),
 ('hua', 2, 'hua', 1),
 ('ua ', 2, 'ua ', 1),
 ('son', 4, 'son', 1),
 ('on ', 4, 'on ', 3),
 ('n 1', 3, 'n 1', 3),
 ('te ', 1, 'te ', 1),
 ('193', 2, '193', 3),
 ('6##', 2, '6##', 1),
 ('##e', 3, '##e', 2),
 ('#em', 2, '#em', 1),
 ('emm', 2, 'emm', 1),
 ('mme', 2, 'mme', 1),
 ('ers', 2, 'ers', 1),
 ('n l', 2, 'n l', 1),
 ('k 1', 2, 'k 1', 1),
 ('211', 2, '211', 1),
 ('129', 2, '129', 1),
 ('29#', 2, '29#', 1),
 ('##a', 1, '##a', 1),
 ('ale', 1, 'ale', 1),
 ('exa', 1, 'exa', 1),
 ('xan', 1, 'xan', 1),
 ('ndr', 1, 'ndr', 1),
 ('ra ', 1, 'ra ', 1),
 ('a g', 2, 'a g', 2),
 (' gr', 2, ' gr', 2),
 ('ros', 1, 'ros', 1),
 ('r 1', 2, 'r 1', 1),
 ('197', 3, '197', 1),
 ('030', 1, '030', 1),
 ('05#', 1, '05#', 1),
 ('5##', 1, '5##', 1),
 ('##m', 2, '##m', 1),
 ('#mi', 2, '#mi', 1),
 ('mic', 2, 'mic', 1),
 ('ich', 2, 'ich', 1),
 ('hae', 2, 'hae', 1),
 ('ael', 2, 'ael', 1),
 ('el ', 2,

In [162]:
con.execute("drop view if exists df").execute(
    "create view df as "
    "select tk1 as token, df1 * df2 as df "
    "from full_outer_df "
    "where tk1 is not null "
    "and tk2 is not null"
).execute("select * from df").fetchall()

[('##j', 3),
 ('#jo', 3),
 ('jos', 2),
 ('osh', 2),
 ('shu', 2),
 ('hua', 2),
 ('ua ', 2),
 ('son', 4),
 ('on ', 12),
 ('n 1', 9),
 ('te ', 1),
 ('193', 6),
 ('6##', 2),
 ('##e', 6),
 ('#em', 2),
 ('emm', 2),
 ('mme', 2),
 ('ers', 2),
 ('n l', 2),
 ('k 1', 2),
 ('211', 2),
 ('129', 2),
 ('29#', 2),
 ('##a', 1),
 ('ale', 1),
 ('exa', 1),
 ('xan', 1),
 ('ndr', 1),
 ('ra ', 1),
 ('a g', 4),
 (' gr', 4),
 ('ros', 1),
 ('r 1', 2),
 ('197', 3),
 ('030', 1),
 ('05#', 1),
 ('5##', 1),
 ('##m', 2),
 ('#mi', 2),
 ('mic', 2),
 ('ich', 2),
 ('hae', 2),
 ('ael', 2),
 ('el ', 2),
 ('sch', 4),
 ('ch ', 2),
 ('h 1', 2),
 ('190', 6),
 ('901', 2),
 ('loy', 1),
 (' sc', 1),
 ('chu', 1),
 ('hue', 1),
 ('etz', 1),
 ('tz ', 1),
 ('z 1', 1),
 ('194', 1),
 ('944', 1),
 ('440', 1),
 ('409', 2),
 ('090', 2),
 ('909', 2),
 ('09#', 2),
 ('ree', 1),
 ('een', 1),
 ('##o', 1),
 ('#ol', 1),
 ('oli', 1),
 ('liv', 1),
 ('ivi', 1),
 ('via', 1),
 ('ia ', 1),
 ('a h', 2),
 (' ho', 1),
 ('hob', 1),
 ('obs', 1),
 ('bso', 1)

In [163]:
widows1 = con.execute(
    "select count(*) "
    "from full_outer_df "
    "where tk2 is null"
).fetchall()[0][0]

widows2 = con.execute(
    "select count(*) "
    "from full_outer_df "
    "where tk1 is null"
).fetchall()[0][0]

R, S = (l_table, r_table) if widows1 > widows2 else (r_table, l_table)
R, S

('src1', 'src2')

In [164]:
con.execute("drop view if exists tkdf").execute(
    "create view tkdf as "
    "select tokens.* "
    ", row_number() OVER (PARTITION BY rid ORDER BY df, tokens.token) as pos "
    "from tokens "
    "join df on tokens.token = df.token"
).execute("select * from tkdf").fetchall()

[('src1', 'S1_0', 26, 'hua', 1),
 ('src1', 'S1_0', 26, 'jos', 2),
 ('src1', 'S1_0', 26, 'osh', 3),
 ('src1', 'S1_0', 26, 'shu', 4),
 ('src1', 'S1_0', 26, 'ua ', 5),
 ('src1', 'S1_0', 26, '##j', 6),
 ('src1', 'S1_0', 26, '#jo', 7),
 ('src1', 'S1_0', 26, '112', 8),
 ('src1', 'S1_0', 26, 'son', 9),
 ('src1', 'S1_0', 26, 'n 1', 10),
 ('src1', 'S1_0', 26, 'on ', 11),
 ('src1', 'S1_0', 26, ' 19', 12),
 ('src1', 'S1_1', 23, 'te ', 1),
 ('src1', 'S1_1', 23, '6##', 2),
 ('src1', 'S1_1', 23, '##j', 3),
 ('src1', 'S1_1', 23, '#jo', 4),
 ('src1', 'S1_1', 23, '112', 5),
 ('src1', 'S1_1', 23, '193', 6),
 ('src1', 'S1_1', 23, ' 19', 7),
 ('src1', 'S1_10', 28, ' do', 1),
 ('src1', 'S1_10', 28, '#el', 2),
 ('src1', 'S1_10', 28, 'abe', 3),
 ('src1', 'S1_10', 28, 'bet', 4),
 ('src1', 'S1_10', 28, 'dom', 5),
 ('src1', 'S1_10', 28, 'eli', 6),
 ('src1', 'S1_10', 28, 'en ', 7),
 ('src1', 'S1_10', 28, 'isa', 8),
 ('src1', 'S1_10', 28, 'lis', 9),
 ('src1', 'S1_10', 28, 'mit', 10),
 ('src1', 'S1_10', 28, 'omi',

## Prefix

In [165]:
con.execute("drop view if exists prefixes").execute(
    "create view prefixes as "
    "select src, rid, rlen, token, pos "
    "from ("
    "SELECT * "
    "FROM tkdf "
    f"where src = '{R}' "
    f"and rlen - pos + 1 >= ceil(rlen * {t}) "
    ") union ("
    "SELECT * "
    "FROM tkdf "
    f"where src = '{S}' "
    f"and rlen - pos + 1 >= ceil(rlen * 2 * {t} / (1+{t})) "
    ")"
).execute("select * from prefixes").fetchall()

[('src2', 'S2_0', 26, ' sc', 1),
 ('src2', 'S2_0', 26, '194', 2),
 ('src2', 'S2_0', 26, '440', 3),
 ('src2', 'S2_0', 26, '944', 4),
 ('src2', 'S2_0', 26, 'chu', 5),
 ('src2', 'S2_0', 26, 'etz', 6),
 ('src2', 'S2_0', 26, 'hue', 7),
 ('src2', 'S2_0', 26, 'tz ', 8),
 ('src2', 'S2_0', 26, 'uet', 9),
 ('src2', 'S2_1', 30, '##a', 1),
 ('src2', 'S2_1', 30, '#al', 2),
 ('src2', 'S2_1', 30, '030', 3),
 ('src2', 'S2_1', 30, '05#', 4),
 ('src2', 'S2_1', 30, '305', 5),
 ('src2', 'S2_1', 30, '5##', 6),
 ('src2', 'S2_1', 30, 'ale', 7),
 ('src2', 'S2_1', 30, 'dra', 8),
 ('src2', 'S2_1', 30, 'eno', 9),
 ('src2', 'S2_1', 30, 'exa', 10),
 ('src2', 'S2_1', 30, 'gro', 11),
 ('src2', 'S2_2', 26, ' li', 1),
 ('src2', 'S2_2', 26, '16#', 2),
 ('src2', 'S2_2', 26, '360', 3),
 ('src2', 'S2_2', 26, '816', 4),
 ('src2', 'S2_2', 26, '936', 5),
 ('src2', 'S2_2', 26, 'ier', 6),
 ('src2', 'S2_2', 26, 'l l', 7),
 ('src2', 'S2_2', 26, 'lie', 8),
 ('src2', 'S2_2', 26, '##m', 9),
 ('src2', 'S2_3', 23, 'loy', 1),
 ('src2'

## Matches

In [135]:
con.execute("drop view if exists candset").execute(
    "CREATE VIEW candset AS ("
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    "FROM prefixes pr1, prefixes pr2 "
    "WHERE pr1.token = pr2.token "
    f"and pr1.src = '{l_table}' "
    f"and pr2.src = '{r_table}'"
    # length filter
    f"AND pr1.rlen >= ceil({t} * pr2.rlen)"
    # positional filter
    "AND LEAST((pr1.rlen - pr1.pos + 1), (pr2.rlen - pr2.pos + 1)) >= "
    f"CEIL((pr1.rlen + pr2.rlen) * {t} / (1 + {t})) "
    "GROUP BY pr1.rid, pr2.rid "
    ")"
).execute("select * from candset").fetchall()

[('S1_0', 'S2_5', 2, 7, 2),
 ('S1_1', 'S2_6', 1, 4, 1),
 ('S1_10', 'S2_7', 8, 10, 7),
 ('S1_10', 'S2_5', 7, 2, 1),
 ('S1_11', 'S2_6', 8, 5, 4),
 ('S1_11', 'S2_7', 4, 5, 3),
 ('S1_11', 'S2_1', 5, 9, 1),
 ('S1_2', 'S2_3', 8, 8, 7),
 ('S1_3', 'S2_1', 9, 10, 9),
 ('S1_4', 'S2_2', 1, 9, 1),
 ('S1_4', 'S2_5', 3, 5, 1),
 ('S1_5', 'S2_3', 9, 8, 8),
 ('S1_6', 'S2_0', 7, 7, 7),
 ('S1_7', 'S2_5', 7, 8, 7),
 ('S1_8', 'S2_4', 9, 9, 9),
 ('S1_9', 'S2_2', 9, 9, 9)]

In [136]:
con.execute(f"drop view if exists {out_table_name}").execute(
    f"create view {out_table_name} as "
    "select r1.rid as rid1, r2.rid as rid2 "
    "from tkdf r1, tkdf r2, candset c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos > maxPos1 "
    "and r2.pos > maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap >= (r1.rlen + r2.rlen) * {t} / (1+{t})"
).execute(f"select * from {out_table_name}").fetchall()

[('S1_10', 'S2_7'),
 ('S1_3', 'S2_1'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4'),
 ('S1_9', 'S2_2')]

# Self-join

In [188]:
# brute force join, to get the gold standard
con.execute("drop view if exists bfjoin").execute(
    "create view bfjoin as "
    "select r1.rid as rid1, r2.rid as rid2, count(*) as overlap "
    "from tokens as r1, tokens as r2 "
    "where r1.token = r2.token "
    "and r1.rid < r2.rid "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
).execute("select * from bfjoin").fetchall()

[('S1_0', 'S3_9', 22),
 ('S1_2', 'S1_5', 22),
 ('S1_3', 'S2_1', 20),
 ('S1_5', 'S2_3', 16),
 ('S1_6', 'S2_0', 17),
 ('S1_7', 'S3_7', 21),
 ('S1_8', 'S2_4', 24),
 ('S1_9', 'S2_2', 23),
 ('S1_10', 'S2_7', 20),
 ('S1_4', 'S3_1', 25),
 ('S1_6', 'S3_5', 17),
 ('S1_11', 'S3_11', 23),
 ('S2_0', 'S3_5', 23),
 ('S2_2', 'S3_3', 21),
 ('S2_6', 'S3_4', 19),
 ('S1_5', 'S3_0', 25),
 ('S1_2', 'S3_0', 22),
 ('S1_8', 'S3_6', 24),
 ('S1_9', 'S3_3', 18),
 ('S2_3', 'S3_0', 16),
 ('S2_4', 'S3_6', 24),
 ('S2_5', 'S3_7', 16)]

## Document frequency

In [172]:
con.execute("drop view if exists df").execute(
    "CREATE VIEW df AS "
    "SELECT token, count(*) AS df "
    "FROM tokens "
    "GROUP BY token "
    # ORDER BY later!
)

In [173]:
con.execute("SELECT * FROM df").fetchall()

[('##e', 6),
 ('#em', 4),
 ('emm', 4),
 ('mme', 4),
 ('mer', 4),
 ('ers', 5),
 ('rso', 3),
 ('son', 7),
 ('on ', 10),
 ('n l', 4),
 (' lo', 4),
 ('loy', 3),
 ('oyc', 2),
 ('ck ', 3),
 ('k 1', 4),
 (' 19', 30),
 ('192', 3),
 ('921', 3),
 ('211', 4),
 ('111', 3),
 ('112', 7),
 ('129', 4),
 ('29#', 4),
 ('##m', 4),
 ('#mi', 4),
 ('mic', 5),
 ('ich', 5),
 ('che', 1),
 ('hel', 1),
 ('el ', 5),
 ('l w', 2),
 (' wu', 2),
 ('wuc', 2),
 ('uch', 2),
 ('cha', 7),
 ('hat', 2),
 ('ats', 2),
 ('tsc', 2),
 ('sch', 7),
 ('ch ', 5),
 ('h 1', 4),
 ('191', 4),
 ('919', 2),
 ('190', 8),
 ('901', 5),
 ('011', 6),
 ('110', 4),
 ('10#', 5),
 ('0##', 5),
 ('##l', 1),
 ('#li', 1),
 ('lie', 3),
 ('ier', 3),
 ('rsc', 2),
 ('h m', 1),
 (' mi', 1),
 ('hae', 4),
 ('ael', 4),
 ('l 1', 1),
 ('193', 6),
 ('936', 3),
 ('360', 3),
 ('608', 6),
 ('081', 7),
 ('816', 3),
 ('16#', 3),
 ('6##', 4),
 ('##c', 2),
 ('#ch', 2),
 ('har', 2),
 ('arl', 2),
 ('rlo', 2),
 ('lot', 2),
 ('ott', 2),
 ('tte', 2),
 ('te ', 3),
 ('e h', 2

In [179]:
con.execute("drop view if exists tkdf").execute(
    "CREATE VIEW tkdf AS "
    "select rid, rlen, tokens.token "
    ", row_number() OVER (PARTITION BY rid ORDER BY df, tokens.token) as pos "
    #"from ("
    #"SELECT tt.*, df.df "
    #"FROM tokens AS tt "
    #"JOIN doc_frequency AS df ON tt.token = df.token "
    #")"
    "from tokens, df "
    "where tokens.token = df.token"
).execute("select * from tkdf").fetchall()

[('S1_0', 26, 'iso', 1),
 ('S1_0', 26, 'ris', 2),
 ('S1_0', 26, ' mo', 3),
 ('S1_0', 26, '101', 4),
 ('S1_0', 26, '123', 5),
 ('S1_0', 26, '23#', 6),
 ('S1_0', 26, '3##', 7),
 ('S1_0', 26, '910', 8),
 ('S1_0', 26, 'a m', 9),
 ('S1_0', 26, 'mor', 10),
 ('S1_0', 26, 'orr', 11),
 ('S1_0', 26, 'rri', 12),
 ('S1_0', 26, '191', 13),
 ('S1_0', 26, 'hua', 14),
 ('S1_0', 26, 'jos', 15),
 ('S1_0', 26, 'osh', 16),
 ('S1_0', 26, 'shu', 17),
 ('S1_0', 26, 'ua ', 18),
 ('S1_0', 26, '##j', 19),
 ('S1_0', 26, '#jo', 20),
 ('S1_0', 26, '011', 21),
 ('S1_0', 26, '112', 22),
 ('S1_0', 26, 'son', 23),
 ('S1_0', 26, 'n 1', 24),
 ('S1_0', 26, 'on ', 25),
 ('S1_0', 26, ' 19', 26),
 ('S1_1', 23, ' wh', 1),
 ('S1_1', 23, '126', 2),
 ('S1_1', 23, '26#', 3),
 ('S1_1', 23, '371', 4),
 ('S1_1', 23, '711', 5),
 ('S1_1', 23, '937', 6),
 ('S1_1', 23, 'an ', 7),
 ('S1_1', 23, 'dan', 8),
 ('S1_1', 23, 'hit', 9),
 ('S1_1', 23, 'jor', 10),
 ('S1_1', 23, 'n w', 11),
 ('S1_1', 23, 'ord', 12),
 ('S1_1', 23, 'rda', 13),
 ('S

## Prefix

In [182]:
con.execute("drop view if exists prefixes").execute(
    "create view prefixes as "
    "select rid, rlen, token, pos "
    "from tkdf "
    f"where rlen - pos + 1 >= ceil(rlen * {t}) "
).execute("select * from prefixes").fetchall()

[('S1_0', 26, 'iso', 1),
 ('S1_0', 26, 'ris', 2),
 ('S1_0', 26, ' mo', 3),
 ('S1_0', 26, '101', 4),
 ('S1_0', 26, '123', 5),
 ('S1_0', 26, '23#', 6),
 ('S1_0', 26, '3##', 7),
 ('S1_0', 26, '910', 8),
 ('S1_0', 26, 'a m', 9),
 ('S1_0', 26, 'mor', 10),
 ('S1_0', 26, 'orr', 11),
 ('S1_0', 26, 'rri', 12),
 ('S1_0', 26, '191', 13),
 ('S1_0', 26, 'hua', 14),
 ('S1_1', 23, ' wh', 1),
 ('S1_1', 23, '126', 2),
 ('S1_1', 23, '26#', 3),
 ('S1_1', 23, '371', 4),
 ('S1_1', 23, '711', 5),
 ('S1_1', 23, '937', 6),
 ('S1_1', 23, 'an ', 7),
 ('S1_1', 23, 'dan', 8),
 ('S1_1', 23, 'hit', 9),
 ('S1_1', 23, 'jor', 10),
 ('S1_1', 23, 'n w', 11),
 ('S1_1', 23, 'ord', 12),
 ('S1_10', 28, '810', 1),
 ('S1_10', 28, '908', 2),
 ('S1_10', 28, 'ett', 3),
 ('S1_10', 28, 'ten', 4),
 ('S1_10', 28, 'tt ', 5),
 ('S1_10', 28, ' do', 6),
 ('S1_10', 28, '#el', 7),
 ('S1_10', 28, 'abe', 8),
 ('S1_10', 28, 'bet', 9),
 ('S1_10', 28, 'dom', 10),
 ('S1_10', 28, 'eli', 11),
 ('S1_10', 28, 'isa', 12),
 ('S1_10', 28, 'ite', 13),


## Matches

In [184]:
con.execute("drop view if exists candset").execute(
    "CREATE VIEW candset AS ("
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    "FROM prefixes pr1, prefixes pr2 "
    "WHERE pr1.rid < pr2.rid "
    "AND pr1.token = pr2.token "
    # length filter
    f"AND pr1.rlen >= ceil({t} * pr2.rlen)"
    # prefix filter
    f"AND pr1.rlen - pr1.pos + 1 >= CEIL(pr1.rlen * 2 * {t} / (1+{t})) "
    # positional filter
    "AND LEAST((pr1.rlen - pr1.pos + 1), (pr2.rlen - pr2.pos + 1)) >= "
    f"CEIL((pr1.rlen + pr2.rlen) * {t} / (1 + {t})) "
    "GROUP BY pr1.rid, pr2.rid "
    ")"
).execute("select * from candset").fetchall()

[('S1_0', 'S3_9', 7, 9, 5),
 ('S1_10', 'S2_7', 10, 11, 5),
 ('S1_11', 'S3_11', 10, 10, 6),
 ('S1_2', 'S3_0', 8, 9, 6),
 ('S1_3', 'S2_1', 8, 11, 2),
 ('S1_4', 'S3_1', 7, 9, 7),
 ('S1_5', 'S3_0', 9, 9, 9),
 ('S1_7', 'S3_7', 9, 7, 6),
 ('S1_8', 'S3_6', 9, 9, 9),
 ('S1_9', 'S2_2', 9, 7, 6),
 ('S1_9', 'S3_3', 8, 9, 3),
 ('S2_0', 'S3_5', 9, 9, 6),
 ('S2_2', 'S3_3', 6, 9, 4),
 ('S2_3', 'S3_0', 8, 9, 2),
 ('S2_4', 'S3_6', 9, 9, 9),
 ('S2_5', 'S3_7', 8, 5, 3),
 ('S2_6', 'S3_4', 5, 10, 3),
 ('S1_2', 'S1_5', 8, 9, 6),
 ('S1_5', 'S2_3', 9, 8, 2),
 ('S1_6', 'S3_5', 7, 9, 1),
 ('S1_7', 'S2_5', 7, 8, 2),
 ('S1_8', 'S2_4', 9, 9, 9),
 ('S1_2', 'S2_3', 8, 8, 1),
 ('S1_6', 'S2_0', 7, 9, 1)]

In [190]:
con.execute(f"drop view if exists {out_table_name}").execute(
    f"create view {out_table_name} as "
    "select r1.rid as rid1, r2.rid as rid2 "
    "from tkdf r1, tkdf r2, candset c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos > maxPos1 "
    "and r2.pos > maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap >= (r1.rlen + r2.rlen) * {t} / (1+{t})"
).execute(f"select * from {out_table_name}").fetchall()

[('S1_0', 'S3_9'),
 ('S1_10', 'S2_7'),
 ('S1_11', 'S3_11'),
 ('S1_2', 'S3_0'),
 ('S1_3', 'S2_1'),
 ('S1_4', 'S3_1'),
 ('S1_5', 'S3_0'),
 ('S1_6', 'S3_5'),
 ('S1_7', 'S3_7'),
 ('S1_8', 'S3_6'),
 ('S1_9', 'S3_3'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S2_2', 'S3_3'),
 ('S2_3', 'S3_0'),
 ('S2_4', 'S3_6'),
 ('S2_5', 'S3_7'),
 ('S2_6', 'S3_4'),
 ('S1_2', 'S1_5'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4')]

# Test

In [191]:
con.execute(
    "select * "
    f"from {out_table_name} m "
    "full outer join bfjoin b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[]